In [1]:
#pip install scikit-surprise

from surprise import SVD
from surprise import KNNBaseline
from surprise import Dataset
from surprise import Reader
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
moviesDF = pd.read_csv("ml-latest-small/movies.csv", sep=",")
moviesDF.movieId = moviesDF.movieId.astype(int)
moviesDF.set_index("movieId", inplace=True)

In [4]:
# simplification of the evaluation case: predict last-k for each user
dfTrain = pd.DataFrame()
dfTest = pd.DataFrame()
for currUser in df.userId.unique():
    dataCurrUser = df[df.userId == currUser]
    currUserTrain = dataCurrUser.iloc[:-10]
    currUserTest = dataCurrUser.iloc[-10:]
    dfTrain = dfTrain.append(currUserTrain)
    dfTest = dfTest.append(currUserTest)

In [5]:
allObjects = df.movieId.unique().astype(str)
allTestSetUsers = dfTest.userId.unique().astype(str)
allObjects

array(['1', '3', '6', ..., '160836', '163937', '163981'], dtype='<U21')

In [6]:
dfTrain.to_csv('ml-latest-small/ratingsTrain.csv', index=False)
dfTest.to_csv('ml-latest-small/ratingsTest.csv', index=False)

In [7]:
file_path = 'ml-latest-small/ratingsTrain.csv'
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)

data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()

In [8]:
trainset

In [47]:
def evaluate(perUserRecommendations):
    #implement evaluation metrics here
    # some accuracy metric is a baseline (precision@k, nDCG, MAP,...)
    # then implement some beyond-accuracy metric (diversity, novelty, coverage, popularity bias,...)    
    # some metrics already implemented somewhere:-) 
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ndcg_score.html
    pass

def recommend(perUserPredictions, top_k):
    # select which items should be recommended
    # baseline is selection of top-k items with highest estimated rating
    # you can implement some diversity / novelty / coverage enhancements here
    pass

def metricStatistics(perUserMetrics):
    # aggregate per-user metrics into an overall statistic
    # baseline is mean, but you can be more creative
    # one other option (needs results of all hyperparam settings) is to compare how many times the algorithm provided better / worse recommendation than other alternatives
    pass

def pickBestVariant(results):
    # based on the results of the evaluation, select best-performing method
    # do the selection based on individual metrics as baseline
    # or think about how to make an aggregated selection based on multiple metrics
    # ideally, visualize the results to see the tradeoff between metrics
    pass

In [38]:
# todo use some hyperparameter tuning loop here
# ideally, try more than one algorithm

algSVD = SVD()
algSVD.fit(trainset)

for uid in allTestSetUsers:
    perUserPredictions = []
    for oid in allObjects:
        
        perUserPredictions.append(algSVD.predict(uid,oid, clip=False))
    
    recs = recommend(perUserPredictions, 20) 
    
    #(m1, m2) = evaluate(recs)
    #it may be necessary to collect additional information for evaluate() e.g. known ratings similarity matrix etc.
    #accumulate m1, m2 to sth. like metricsPerUser

results = metricStatistics(metricsPerUser)



[Prediction(uid='1', iid='1', r_ui=None, est=4.568659561709939, details={'was_impossible': False}),
 Prediction(uid='1', iid='3', r_ui=None, est=4.256257950010596, details={'was_impossible': False}),
 Prediction(uid='1', iid='6', r_ui=None, est=4.515272473512568, details={'was_impossible': False}),
 Prediction(uid='1', iid='47', r_ui=None, est=4.9334314924467595, details={'was_impossible': False}),
 Prediction(uid='1', iid='50', r_ui=None, est=5.016741830046825, details={'was_impossible': False}),
 Prediction(uid='1', iid='70', r_ui=None, est=4.096296011864284, details={'was_impossible': False}),
 Prediction(uid='1', iid='101', r_ui=None, est=4.691775398434401, details={'was_impossible': False}),
 Prediction(uid='1', iid='110', r_ui=None, est=4.608598750397419, details={'was_impossible': False}),
 Prediction(uid='1', iid='151', r_ui=None, est=4.586766452538002, details={'was_impossible': False}),
 Prediction(uid='1', iid='157', r_ui=None, est=3.986362400433419, details={'was_impossible

In [26]:

algo.predict("1","2", clip=False)

Prediction(uid='1', iid='2', r_ui=None, est=4.268015663114385, details={'was_impossible': False})

In [51]:
# Use the knn.
algoKNN = KNNBaseline()
algoKNN.fit(trainset)
pred = algoKNN.predict("1","2", clip=False)
print(pred)
print(pred.uid, pred.iid, pred.est)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
user: 1          item: 2          r_ui = None   est = 4.15   {'actual_k': 40, 'was_impossible': False}
1 2 4.148609180175668
